<a href="https://colab.research.google.com/github/Beatriz-Yun/CatchingDudeoji/blob/main/ch08/8.3%20%EC%9E%AC%ED%98%95%EC%84%B1%EA%B3%BC%20%ED%94%BC%EB%B2%97.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 8장. 데이터 준비하기

## 8.3절 재형성과 피벗 (Reshaping and Pivot)
(334-344쪽)

**표 형식의 데이터를 재배치**하는 기본적인 연산을 **'재형성' 또는 '피벗' 연산**이라고 한다.

## 8.3.1 계층적 색인으로 재형성하기 (Reshaping with Hierarchical Indexing)

계층적 색인은 DataFrame의 데이터를 재배치하는 2가지 방식을 제공한다.
- **stack**: 데이터의 칼럼을 로우로 pivot(회전)시킨다.
- **unstack**: 데이터의 로우를 칼럼으로 pivot(회전)시킨다.

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data = pd.DataFrame(np.arange(6).reshape((2, 3)),
                    index=pd.Index(['Ohio', 'Colorado'], name='state'),         # 인덱스명
                    columns=pd.Index(['one', 'two', 'three'], name='number'))   # 칼럼명
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


**stack메서드**: 칼럼이 로우로 pivot된다. Series객체를 반환함.
- one, two, three가 'state'의 하위 수준 로우로 들어간다.

[공식문서) pandas.DataFrame.stack](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.stack.html)

In [ ]:
result = data.stack()
result    # Series 객체

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int64

**unstack메서드**: 로우가 칼럼으로 pivot된다. 계층적 색인을 가진 Series로부터 DataFrame을 얻을 수 있다.


[공식문서) pandas.DataFrame.unstack](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.unstack.html)

In [ ]:
result.unstack()

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


#### stack과 unstack메서드의 **매개변수 level**은 어떤 걸 먼저 끄집어내는지 지정하는 인자이다.
- level의 default값은 -1로, 기본적으로 **가장 안쪽에 있는 level부터** 끄집어낸다. (위의 경우에서 가장 안쪽에 있는 level은 'number'이다.)
- **level 숫자나 이름**을 전달하여 끄집어내는 단계를 지정할 수 있다.

In [ ]:
result.unstack(0)    # level이 0인 인덱스는 상위계층색인 'state'이다.

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [ ]:
result.unstack('state')    # 먼저 끄집어낼 level 이름을 전달하였다.

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


#### 해당 level에 있는 모든 값이 하위 그룹에 속하지 않을 경우, unstack을 하게 되면 누락된 데이터가 생길 수 있다.
- 'one'의 하위그룹은 ['a', 'b', **'c', 'd'**]
- 'two'의 하위그룹은 [**'c', 'd'**, 'e']

<br>

'one'과 'two'의 level에 있는 모든 값이 하위 그룹에 속하지 않는다!!<br>
👉 unstack을 하게 되면 'a', 'b', 'c'에 누락된 데이터가 생긴다.

In [ ]:
s1 = pd.Series([0, 1, 2, 3], index=['a', 'b', 'c', 'd'])
s2 = pd.Series([4, 5, 6], index=['c', 'd', 'e'])
data2 = pd.concat([s1, s2], keys=['one', 'two'])    # Series객체인 s1은 'one'인덱스에, s2는 'two'인덱스에 속하게 된다. (계층적 색인)

data2

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: int64

In [ ]:
data2.unstack()    # 안쪽 level 먼저 끄집어 냄.

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


stack 메서드는 누락된 데이터를 자동으로 걸러내기 때문에 **연산을 쉽게 원상 복구**할 수 있다.

In [ ]:
data2.unstack().stack()    # NaN이 자동으로 제거된다.

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
two  c    4.0
     d    5.0
     e    6.0
dtype: float64

In [ ]:
data2.unstack().stack(dropna=False)    # dropna 매개변수로 NaN을 그대로 남길 수 있다.

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
     e    NaN
two  a    NaN
     b    NaN
     c    4.0
     d    5.0
     e    6.0
dtype: float64

DataFrame을 unstack할 때 unstack level은 결과에서 가장 낮은 단계가 된다.(?)

In [ ]:
df = pd.DataFrame({'left': result, 'right': result + 5},
                  columns=pd.Index(['left', 'right'], name='side'))
df

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [ ]:
df.unstack('state')    # 'state'를 칼럼으로 끄집어냄.
                       # 'side'보다 밑에 있게 된다. 'state'는 'side'의 하위 수준 색인.

side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

stack을 호출할 때 쌓을 축의 이름을 지정할 수 있다.

In [ ]:
df.unstack('state').stack('side')    # 위의 unstack결과를 'side'로 다시 stack

state         Colorado  Ohio
number side                 
one    left          3     0
       right         8     5
two    left          4     1
       right         9     6
three  left          5     2
       right        10     7

## 8.3.2 긴 형식에서 넓은 형식으로 피벗하기 (Pivoting “Long” to “Wide” Format)

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/macrodata.csv')
data.head()

,year,quarter,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
0,1959.0,1.0,2710.349,1707.4,286.898,470.045,1886.9,28.98,139.7,2.82,5.8,177.146,0.00,0.00
1,1959.0,2.0,2778.801,1733.7,310.859,481.301,1919.7,29.15,141.7,3.08,5.1,177.830,2.34,0.74
2,1959.0,3.0,2775.488,1751.8,289.226,491.260,1916.4,29.35,140.5,3.82,5.3,178.657,2.74,1.09
3,1959.0,4.0,2785.204,1753.7,299.356,484.052,1931.3,29.37,140.0,4.33,5.6,179.386,0.27,4.06
4,1960.0,1.0,2847.699,1770.5,331.722,462.199,1955.5,29.54,139.6,3.50,5.2,180.007,2.31,1.19


In [ ]:
periods = pd.PeriodIndex(year=data.year, quarter=data.quarter, name='date')    # 시간간격을 나타내는 자료형. 연도와 분기 칼럼을 합친다.
columns = pd.Index(['realgdp', 'infl', 'unemp'], name='item')

data = data.reindex(columns=columns)
data.index = periods.to_timestamp('D', 'end')

ldata = data.stack().reset_index().rename(columns={0: 'value'})

In [ ]:
periods    # 11장 참고 - pd.PeriodIndex

PeriodIndex(['1959Q1', '1959Q2', '1959Q3', '1959Q4', '1960Q1', '1960Q2',
             '1960Q3', '1960Q4', '1961Q1', '1961Q2',
             ...
             '2007Q2', '2007Q3', '2007Q4', '2008Q1', '2008Q2', '2008Q3',
             '2008Q4', '2009Q1', '2009Q2', '2009Q3'],
            dtype='period[Q-DEC]', name='date', length=203, freq='Q-DEC')

ldata는 **긴 형식의 데이터**이다.
- 여러 시계열이나 둘 이상의 키('date'와 'item)를 가지고 있는 관측 데이터에서 사용한다.
- 각 행은 '단일 관측치'를 나타낸다.


관계형 데이터베이스 관점에서 보면, 테이블에 데이터가 추가되거나 삭제될 때 **'date'와 'item'은 기본키(primary key)**가 된다.<br>

<br>

긴 형식의 데이터로는 작업이 용이하지 않을 수 있어서<br>
하나의 DataFrame에 'date'칼럼의 시간값으로 색인된 개별 item을 칼럼으로 포함시키고 싶어질 것이다.<br>
👉 이런 변형을 지원하는 것이 바로 **DataFrame의 pivot메서드**이다.

In [ ]:
ldata[:10]

,date,item,value
0,1959-03-31 23:59:59.999999999,realgdp,2710.349
1,1959-03-31 23:59:59.999999999,infl,0.000
2,1959-03-31 23:59:59.999999999,unemp,5.800
3,1959-06-30 23:59:59.999999999,realgdp,2778.801
4,1959-06-30 23:59:59.999999999,infl,2.340
5,1959-06-30 23:59:59.999999999,unemp,5.100
6,1959-09-30 23:59:59.999999999,realgdp,2775.488
7,1959-09-30 23:59:59.999999999,infl,2.740
8,1959-09-30 23:59:59.999999999,unemp,5.300
9,1959-12-31 23:59:59.999999999,realgdp,2785.204


pivot메서드의 처음 두 인자는 **로우와 칼럼 인덱스로 사용될 칼럼명**이다.
- 로우 인덱스로 사용: 'date', 칼럼 인덱스로 사용: 'item'

마지막 두 인자는 **DataFrame에 채워 넣을 값을 담고 있는 칼럼**이다.
- 'value'

In [ ]:
pivoted = ldata.pivot('date', 'item', 'value')    # infl, realgdp, unemp는 'item'의 값이다.
pivoted

item,infl,realgdp,unemp
date,,,
1959-03-31 23:59:59.999999999,0.00,2710.349,5.8
1959-06-30 23:59:59.999999999,2.34,2778.801,5.1
1959-09-30 23:59:59.999999999,2.74,2775.488,5.3
1959-12-31 23:59:59.999999999,0.27,2785.204,5.6
1960-03-31 23:59:59.999999999,2.31,2847.699,5.2
...,...,...,...
2008-09-30 23:59:59.999999999,-3.16,13324.600,6.0
2008-12-31 23:59:59.999999999,-8.79,13141.920,6.9
2009-03-31 23:59:59.999999999,0.94,12925.410,8.1


#### 한 번에 2개의 칼럼을 동시에 변형할 수 있다.

In [ ]:
ldata['value2'] = np.random.randn(len(ldata))   # 'value2'칼럼 생성
ldata[:10]

,date,item,value,value2
0,1959-03-31 23:59:59.999999999,realgdp,2710.349,0.247819
1,1959-03-31 23:59:59.999999999,infl,0.000,-0.225046
2,1959-03-31 23:59:59.999999999,unemp,5.800,0.033125
3,1959-06-30 23:59:59.999999999,realgdp,2778.801,-1.755287
4,1959-06-30 23:59:59.999999999,infl,2.340,-0.989052
5,1959-06-30 23:59:59.999999999,unemp,5.100,0.518612
6,1959-09-30 23:59:59.999999999,realgdp,2775.488,0.154187
7,1959-09-30 23:59:59.999999999,infl,2.740,0.830718
8,1959-09-30 23:59:59.999999999,unemp,5.300,-1.566133
9,1959-12-31 23:59:59.999999999,realgdp,2785.204,-0.132290


마지막 인자를 생략해서 계층적 칼럼을 가지는 DataFrame을 얻을 수 있다.

In [ ]:
pivoted = ldata.pivot('date', 'item')
pivoted[:5]

value            ...    value2          
item                           infl   realgdp  ...   realgdp     unemp
date                                           ...                    
1959-03-31 23:59:59.999999999  0.00  2710.349  ...  0.247819  0.033125
1959-06-30 23:59:59.999999999  2.34  2778.801  ... -1.755287  0.518612
1959-09-30 23:59:59.999999999  2.74  2775.488  ...  0.154187 -1.566133
1959-12-31 23:59:59.999999999  0.27  2785.204  ... -0.132290  0.482726
1960-03-31 23:59:59.999999999  2.31  2847.699  ...  0.203997  0.854384

[5 rows x 6 columns]

각 피봇테이블을 위의 계층적 칼럼을 가지는 DataFrame을 인덱싱하여 얻을 수 있다.

In [ ]:
pivoted['value'][:5]

item,infl,realgdp,unemp
date,,,
1959-03-31 23:59:59.999999999,0.00,2710.349,5.8
1959-06-30 23:59:59.999999999,2.34,2778.801,5.1
1959-09-30 23:59:59.999999999,2.74,2775.488,5.3
1959-12-31 23:59:59.999999999,0.27,2785.204,5.6
1960-03-31 23:59:59.999999999,2.31,2847.699,5.2


#### pivot은 단지 **set_index를 사용해서 계층적 색인을 만들고, unstack메서드를 사용해서 형태를 변경**하는 단축키 같은 메서드이다.

In [ ]:
ldata.set_index(['date', 'item'])

value    value2
date                          item                        
1959-03-31 23:59:59.999999999 realgdp   2710.349  0.247819
                              infl         0.000 -0.225046
                              unemp        5.800  0.033125
1959-06-30 23:59:59.999999999 realgdp   2778.801 -1.755287
                              infl         2.340 -0.989052
...                                          ...       ...
2009-06-30 23:59:59.999999999 infl         3.370 -1.856087
                              unemp        9.200  1.241295
2009-09-30 23:59:59.999999999 realgdp  12990.341  0.605301
                              infl         3.560  0.129146
                              unemp        9.600  0.727602

[609 rows x 2 columns]

계층적 칼럼을 가지는 DataFrame이 만들어진다.

In [ ]:
unstacked = ldata.set_index(['date', 'item']).unstack('item')    # set_index와 unstack을 순차적으로 적용한 결과와 pivot의 결과는 같다.
unstacked[:7]

value            ...    value2          
item                           infl   realgdp  ...   realgdp     unemp
date                                           ...                    
1959-03-31 23:59:59.999999999  0.00  2710.349  ...  0.247819  0.033125
1959-06-30 23:59:59.999999999  2.34  2778.801  ... -1.755287  0.518612
1959-09-30 23:59:59.999999999  2.74  2775.488  ...  0.154187 -1.566133
1959-12-31 23:59:59.999999999  0.27  2785.204  ... -0.132290  0.482726
1960-03-31 23:59:59.999999999  2.31  2847.699  ...  0.203997  0.854384
1960-06-30 23:59:59.999999999  0.14  2834.390  ... -0.518109  1.004842
1960-09-30 23:59:59.999999999  2.70  2839.022  ... -0.644334 -1.786207

[7 rows x 6 columns]

## 8.3.3 넓은 형식에서 긴 형식으로 피벗하기 (Pivoting “Wide” to “Long” Format)
pivot과 반대되는 연산은 **pandas의 melt**이다.
- 여러 칼럼을 하나로 병합하고 DataFrame을 입력보다 긴 형태로 만들어낸다.

[공식문서) pandas.melt](https://pandas.pydata.org/docs/reference/api/pandas.melt.html)

In [ ]:
df = pd.DataFrame({'key': ['foo', 'bar', 'baz'],
                   'A': [1, 2, 3],
                   'B': [4, 5, 6],
                   'C': [7, 8, 9]})
df

,key,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9


pandas의 melt메서드를 사용할 때는 어떤 컬럼을 그룹 구분자로 사용할 것인지 지정할 수 있다.
- 'key'를 그룹 구분자로 지정해보자.
- **의미) 'key'를 기준으로 A,B,C를 행으로 녹인다.(melt)**
- A,B,C값을 인덱스 별로 추출하기에 용이해진다.

In [ ]:
melted = pd.melt(df, ['key'])     # 데이터가 길어짐.
melted

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
6,foo,C,7
7,bar,C,8
8,baz,C,9


#### pivot을 이용해서 원래대로 되돌릴 수 있다.

In [ ]:
reshaped = melted.pivot('key', 'variable', 'value')
reshaped

variable,A,B,C
key,,,
bar,2,5,8
baz,3,6,9
foo,1,4,7


pivot의 결과는 로우 라벨로 사용하던 칼럼에서 인덱스를 생성하므로 reset_index()를 사용해서 데이터를 다시 칼럼으로 돌려놓는다.

In [ ]:
reshaped.reset_index()

variable,key,A,B,C
0,bar,2,5,8
1,baz,3,6,9
2,foo,1,4,7


#### 데이터 값으로 사용할 칼럼들의 집합을 지정할 수도 있다.
- id_var, value_vars 매개변수 사용
- 'key'를 기준으로 'A'와 'B'의 값을 행에 녹인다.

In [ ]:
pd.melt(df, id_vars=['key'], value_vars=['A', 'B'])

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6


#### pandas.melt는 그룹 구분자 없이도 사용할 수 있다.
A, B, C와 값이 행으로 들어간다.

In [ ]:
pd.melt(df, value_vars=['A', 'B', 'C'])

,variable,value
0,A,1
1,A,2
2,A,3
3,B,4
4,B,5
5,B,6
6,C,7
7,C,8
8,C,9


In [ ]:
pd.melt(df, value_vars=['key', 'A', 'B'])

,variable,value
0,key,foo
1,key,bar
2,key,baz
3,A,1
4,A,2
5,A,3
6,B,4
7,B,5
8,B,6
